## This script is used for the prepraring dataset for the finetuning DNABERT for SARS-CoV-2 virant clasiification 

In [1]:
import os, tarfile
import glob, time, csv, zipfile, gzip
import pandas as pd
import itertools as it
import sys

In [2]:
os.getcwd()

'/mnt/d/GitHub/Postdoc/preprocessing'

In [3]:
variant_dict = {
    "alpha": 0,
    "beta": 1,
    "delta": 2,
    "gamma": 3,
    "eta":4,
    "iota":5,
    "kappa":6,
    "lambda":7,
    "zeta":8
}

In [4]:
variant_dict['alpha']

0

In [5]:
#For Mac
#parent_dir_path = "/Users/pratik/Box/Datasets/work_dir/SARS_finetuned/"
#For Windows
parent_dir_path = "/home/pratik/Work_directory/SARS_finetuned/GISAID"
os.chdir(parent_dir_path)

### Function for creating non-overlappingg chunks with starting position zero and other mentioned starting position

In [6]:
def create_non_overlap_genome(sequence, chunk_size):
    chunks = []
    starting_position_list = [0, 1000, 2000]
    for position in starting_position_list:
        temp_chunks = [sequence[i:i+chunk_size] for i in range(position, len(sequence), chunk_size)]
        if(len(temp_chunks[-1])<chunk_size):
            del temp_chunks[-1]
        chunks.extend(temp_chunks)
    last_chunk = sequence[-chunk_size:]
    chunks.append(last_chunk)
    return(chunks)

### Finetuning data preparation

In [7]:
def finetune_data_preparation(file_lines, label):
    final_list = []
    for key,group in it.groupby(file_lines,lambda line: line.startswith('>')):
        if not key:
            #count= count+1
            group = list(group)
            stripped_group = list(map(lambda s: s.strip(), group))
            concat_group = ' '.join(stripped_group) 
            concat_group = concat_group.replace(" ","")
            temp_list= create_non_overlap_genome(concat_group, 3000)
            final_list.extend(temp_list)
    temp_df= pd.DataFrame(final_list, columns= ['Sequence'])
    print ("temp", temp_df.shape)
    temp_df = temp_df.drop_duplicates().reset_index(drop=True)
    temp_df['Label']= label
    return (temp_df)

## Remove contradict entries and save in a `.csv` file

In [8]:
def remv_contradict_entries(g, directory):
    print (g.shape)
    df1= g.groupby('Sequence')
    df2= df1.filter(lambda x: len(x) == 1).reset_index(drop=True)
    print (df2.groupby('Label').count())
    df2.to_csv("../Finetuned_SARS_"+directory+".csv", index=False)

In [10]:
for directory in os.listdir():
    count =0
    print (directory)
    input()
    final_df= pd.DataFrame(columns= ['Sequence','Label' ])
    current_path= os.path.join(parent_dir_path,directory)
    #print (current_path)
    folders = os.listdir(directory)
    for subfolder in folders:
        path = os.path.join(current_path, subfolder)
        print (path)
        files= os.listdir(path)
        for file in files:
            if file.endswith("tar"):
                file_path= os.path.join(path, file)
                contents = tarfile.open(file_path)
                #print(contents.getnames())
                for file_name, file_object in zip(contents.getnames(), contents.getmembers()):
                    if file_name.endswith('fasta'):
                        print(file_name)
                        f=contents.extractfile(file_object)
                        file_content=f.readlines()
                        file_content_str = [x.decode() for x in file_content]
                        label = variant_dict[subfolder]
                        temp_df = finetune_data_preparation(file_content_str, label)
                        final_df = final_df.append(temp_df, ignore_index=True)
                        print ("Final",final_df.shape)
    remv_contradict_entries(final_df, directory)

Africa


/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/beta
1629316238802.sequences.fasta
temp (138012, 1)
Final (40785, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/zeta
1629316826903.sequences.fasta
temp (280, 1)
Final (40905, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/eta
1629317297200.sequences.fasta
temp (6412, 1)
Final (44371, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/lambda
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/iota
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/delta
1629316615035.sequences.fasta
temp (29512, 1)
Final (55442, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/kappa
1629317487078.sequences.fasta
temp (252, 1)
Final (55667, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/alpha
1629314973953.sequences.fasta
temp (32004, 1)
Final (66372, 2)
(66372, 2)
       Sequence
Label          
0         10414
1         40358
2         10895
4          3347
6           206
8  

/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/beta
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/zeta
1629323030691.sequences.fasta
temp (45192, 1)
Final (11036, 2)
1629322790999.sequences.fasta
temp (7280, 1)
Final (14221, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/eta
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/lambda
1629322196621.sequences.fasta
temp (1708, 1)
Final (15271, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/iota
1629323180991.sequences.fasta
temp (5712, 1)
Final (17564, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/delta
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/gamma
1629321882420.sequences.fasta
temp (137312, 1)
Final (45723, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/kappa
1629323292487.sequences.fasta
temp (20216, 1)
Final (51836, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/alpha
(51836, 2)
       Sequence
Label          
3         27891
5 

/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/beta
1629323731996.sequences.fasta
temp (1960, 1)
Final (1010, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/zeta
1629323703456.sequences.fasta
temp (140, 1)
Final (1128, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/eta
1629324106221.sequences.fasta
temp (196, 1)
Final (1268, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/lambda
1629324181209.sequences.fasta
temp (28, 1)
Final (1296, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/iota
1629324129472.sequences.fasta
temp (56, 1)
Final (1324, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/delta
1629323796416.sequences.fasta
temp (16100, 1)
Final (6334, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/gamma
1629323757740.sequences.fasta
temp (196, 1)
Final (6442, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/kappa
1629324155720.sequences.fasta
temp (2240, 1)
Final (7108, 2)
/home/prat

/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/beta
1629389299611.sequences.fasta
temp (504, 1)
Final (312, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/zeta
1629389399049.sequences.fasta
temp (69356, 1)
Final (26893, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/eta
1629389807077.sequences.fasta
temp (28, 1)
Final (26921, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/lambda
1629389882085.sequences.fasta
temp (6132, 1)
Final (30425, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/iota
1629389834686.sequences.fasta
temp (10108, 1)
Final (33735, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/delta
1629389326073.sequences.fasta
temp (9604, 1)
Final (37163, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/kappa
1629389859687.sequences.fasta
temp (28, 1)
Final (37191, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/alpha
1629389233623.s

/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/beta
1629319610891.sequences.fasta
temp (63840, 1)
Final (18583, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/zeta
1629319354850.sequences.fasta
temp (44043, 1)
Final (35308, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/eta
1629319246862.sequences.fasta
temp (14028, 1)
Final (39246, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/lambda
1629318739604.sequences.fasta
temp (3528, 1)
Final (41277, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/iota
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/delta
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/kappa
1629319061098.sequences.fasta
temp (11872, 1)
Final (45623, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/alpha
1630017471230.sequences.fasta
temp (21196, 1)
Final (55194, 2)
(55194, 2)
       Sequence
Label          
0          9344
1       

/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/beta
1629321017543.sequences.fasta
temp (56617, 1)
Final (19469, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/zeta
1629320920540.sequences.fasta
temp (504, 1)
Final (19901, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/eta
1629321116083.sequences.fasta
temp (8848, 1)
Final (25713, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/lambda
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/iota
1629321359081.sequences.fasta
temp (476, 1)
Final (26138, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/delta
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/gamma
1629320543386.sequences.fasta
temp (4844, 1)
Final (27813, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/kappa
1629321299449.sequences.fasta
temp (73556, 1)
Final (58077, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/alpha
(58077, 2)
       Sequence
Label          
1         19026
3          1596
4       